In [1]:
import pandas as pd
import os

In [ ]:
'aa_dataset-tickets-multi-lang-5-2-50-version.csv'
'dataset-tickets-multi-lang-4-20k.csv'
'dataset-tickets-multi-lang3-4k.csv'

In [2]:
path = 'aa_dataset-tickets-multi-lang-5-2-50-version.csv'
df = pd.read_csv('../data/data-support-tickets/' + path)
df.shape

(28587, 16)

In [ ]:
"""
    queue - (Очередь) Указывает отдел, в который направляется электронный тикет.
    Priority (Приоритет) - Указывает на срочность и важность проблемы Low | Medium | Critical
    Language - EN, DE, ES, FR, PT
    Subject (Предмет) - Тема письма клиента
    Body (Тело) - Текст электронного письма клиента
    Answer (Отвечать) - Ответ, предоставленный агентом службы поддержки
    Type (Тип) - Тип билета, выбранный агентом
    Business Type (Тип бизнеса) - Тип деятельности службы поддержки
    Tags (Теги) - Теги/категории, присвоенные тикету, разбитые на десять столбцов в наборе данных
    
    Queue(Очередь)
    Указывает отдел, к которому относится тикет по электронной почте. Это помогает направить тикет в соответствующую службу поддержки для разрешения.
    
    💻 Technical Support (Техническая поддержка): технические проблемы и запросы на поддержку.
    🈂️ Customer Service (Обслуживание клиентов): запросы клиентов и заявки на обслуживание.
    💰 Billing and Payments (Выставление счетов и платежи): проблемы с выставлением счетов и обработкой платежей.
    🖥️ Product Support (Поддержка продукта): поддержка по вопросам, связанным с продуктом.
    🌐 IT Support (ИТ-поддержка): внутренние вопросы ИТ-поддержки и инфраструктуры.
    🔄 Returns and Exchanges (Возврат и обмен): Возврат и обмен товара.
    📞 Sales and Pre-Sales (Продажи и предпродажи): вопросы по продажам и предпродажам.
    🧑‍💻 Human Resources (Кадровые ресурсы): запросы сотрудников и вопросы, связанные с кадрами.
    ❌ Service Outages and Maintenance (Перебои в обслуживании и техническое обслуживание): Перебои в обслуживании и техническое обслуживание.
    📮 General Inquiry (Общий запрос): общие вопросы и запросы информации.
    
    Types (Типы)
    Различные типы тикетов классифицированы для понимания характера запросов или проблем.
    
    ❗Incident (Инцидент): Неожиданная проблема, требующая немедленного внимания.
    📝 Request (Запрос): Обычный запрос или запрос на обслуживание.
    ⚠️ Problem (Проблема): Основная проблема, вызывающая множественные инциденты.
    🔄 Change (Изменение): Планируемое изменение или обновление.

    Теги
    Теги/категории, назначаемые тикету, позволяют дополнительно классифицировать и определять общие проблемы или темы.
    
    Примеры: «Поддержка продукта», «Техническая поддержка», «Запрос на продажу».

    
"""


In [3]:
df.head(2)

,subject,body,answer,type,queue,priority,language,version,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
0,Wesentlicher Sicherheitsvorfall,"Sehr geehrtes Support-Team,\n\nich möchte eine...",Vielen Dank für die Meldung des kritischen Sic...,Incident,Technical Support,high,de,51,Security,Outage,Disruption,Data Breach,NaN,NaN,NaN,NaN
1,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...","Thank you for reaching out, <name>. We are awa...",Incident,Technical Support,high,en,51,Account,Disruption,Outage,IT,Tech Support,NaN,NaN,NaN


### Проверка данных + предобработка

In [4]:
columns_incloud = ['subject', 'body', 'queue', 'priority', 'language']

df = df[columns_incloud]

In [5]:
df.isna().sum()

subject     3838
body           0
queue          0
priority       0
language       0
dtype: int64

In [6]:
df.loc[:, 'subject'] = df['subject'].fillna("no subject")
df.loc[:, 'text'] = df['subject'] + " " + df['body']

df['subject'].isna().sum()

np.int64(0)

In [7]:
df = df.drop(['subject', 'body'], axis=1)

In [8]:
df = df[df.language == 'en']

In [9]:
df = df.drop(['language'], axis=1)

In [10]:
df.queue.value_counts()

queue
Technical Support                  4737
Product Support                    3073
Customer Service                   2410
IT Support                         1942
Billing and Payments               1595
Returns and Exchanges               820
Service Outages and Maintenance     664
Sales and Pre-Sales                 513
Human Resources                     348
General Inquiry                     236
Name: count, dtype: int64

In [11]:
df.priority.value_counts()

priority
medium    6618
high      6346
low       3374
Name: count, dtype: int64

In [12]:
df[['priority', 'queue']].value_counts()

priority  queue                          
high      Technical Support                  2761
medium    Product Support                    1575
          Technical Support                  1442
          Customer Service                   1121
high      IT Support                          938
          Product Support                     899
low       Customer Service                    837
medium    IT Support                          826
          Billing and Payments                770
low       Product Support                     599
          Technical Support                   534
high      Billing and Payments                483
          Service Outages and Maintenance     467
          Customer Service                    452
low       Billing and Payments                342
medium    Returns and Exchanges               322
low       Returns and Exchanges               313
medium    Sales and Pre-Sales                 237
low       Sales and Pre-Sales                 188
high    

### Разделение на тестовую и трейновую

In [13]:
df.head()

,queue,priority,text
1,Technical Support,high,"Account Disruption Dear Customer Support Team,..."
2,Returns and Exchanges,medium,Query About Smart Home System Integration Feat...
3,Billing and Payments,low,Inquiry Regarding Invoice Details Dear Custome...
4,Sales and Pre-Sales,medium,Question About Marketing Agency Software Compa...
5,Technical Support,high,"Feature Query Dear Customer Support,\n\nI hope..."


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train_queue, x_valid_queue, y_train_queue, y_valid_queue = train_test_split(df.text, df.queue, test_size=0.2, random_state=42)

x_train_priority, x_valid_priority, y_train_priority, y_valid_priority = train_test_split(df.text, df.priority, test_size=0.2, random_state=42)

### Pipeline

In [ ]:
# построить pipeline + закодировать фичи через tf-idf

In [16]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [17]:
pipeline = Pipeline(steps=[
    ('tf_idf', TfidfVectorizer(max_features=20000, ngram_range=(1, 2), min_df=3)),
    ('model', LogisticRegression(max_iter=1000, solver='lbfgs', C=1))
])

#### Предсказание по queue

In [18]:
pipeline.fit(x_train_queue, y_train_queue)

f1_macro_queue_train = f1_score(y_train_queue, pipeline.predict(x_train_queue), average='macro')

y_pred_queue = pipeline.predict(x_valid_queue)
f1_macro_queue = f1_score(y_valid_queue, y_pred_queue, average='macro')
print(f"Macro F1: train {f1_macro_queue_train:.4f}, test {f1_macro_queue:.4f}")

Macro F1: train 0.5818, test 0.3905


#### Предсказание по priority

In [19]:
pipeline.fit(x_train_priority, y_train_priority)

f1_macro_priority_train = f1_score(y_train_priority, pipeline.predict(x_train_priority), average='macro')

y_pred_priority = pipeline.predict(x_valid_priority)
f1_macro_priority = f1_score(y_valid_priority, y_pred_priority, average='macro')
print(f"Macro F1: train {f1_macro_priority_train:.4f}, test {f1_macro_priority:.4f}")

Macro F1: train 0.8162, test 0.5662


In [20]:
overall_f1 = 0.3 * f1_macro_queue + 0.7 * f1_macro_priority
print(f"Mean Macro F1: {overall_f1:.4f}")

Mean Macro F1: 0.5135


In [21]:
# 2: Комбинированная точность (joint accuracy)
joint_acc = np.mean((y_valid_queue == y_pred_queue) & (y_valid_priority == y_pred_priority))
print(f"Joint accuracy: {joint_acc:.4f}")

Joint accuracy: 0.3464
